In [2]:
import pandas as pd
pd.options.display.max_columns=200
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.pipeline import make_pipeline
import bs4 as bs
import nltk
from nltk.corpus import stopwords, words
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize, wordpunct_tokenize, RegexpTokenizer
from wordcloud import WordCloud
from PIL import Image
from pandarallel import pandarallel

In [3]:
data_bow = pd.read_csv('data_bow.csv')

In [4]:
data_bow.head()

,Id,Body,Title,Tags,text
0,40101130,consider pd.series import panda import numpy n...,calculate rolling idxmax,<python><pandas><numpy><dataframe><series>,calculate rolling idxmax consider pd.series...
1,662383,'ve using time substring str_col patindex str_...,better technique trimming leading zero sql server,<sql><sql-server><sql-server-2005><tsql><string>,better technique trimming leading zero sql ser...
2,662421,trying install java application linux machine ...,x11 display variable mean,<java><linux><variables><x11><headless>,x11 display variable mean trying install ja...
3,3520133,library allows easily conveniently create obje...,object oriented callback,<c++><oop><callback><pointer-to-member><eiffel>,object oriented callback library allows eas...
4,1396164,edit whole question unclear want use openssl.n...,why n't .net find openssl.net dll,<c#><.net><dll><dllimport><dllnotfoundexception>,why n't .net find openssl.net dll edit whol...


In [23]:
#Split the data into train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_bow[['Title','Body','text']], data_bow['Tags'], test_size=0.2, random_state=42)

In [43]:
# création du bag of words (CountVectorizer et Tf-idf)

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cvect = CountVectorizer(stop_words='english', max_df=0.9, min_df=2)
ctf = TfidfVectorizer(stop_words='english', max_df=0.95, min_df=1)

feat = 'Title'
cv_fit = cvect.fit(X_train[feat])
ctf_fit = ctf.fit(X_train[feat])

cv_transform_title = cvect.transform(X_train['Title'])  
ctf_transform_title = ctf.transform(X_train['Title'])

cv_transform_text = cvect.transform(X_train['text'])
ctf_transform_text = ctf.transform(X_train['text'])

In [39]:
cv_transform_title.shape, ctf_transform_title.shape

((40000, 9073), (40000, 19476))

In [44]:
cv_transform_text.shape, ctf_transform_text.shape

((40000, 9073), (40000, 19476))

In [47]:
#type of cv_transform_title
type(cv_transform_title)

pandas.core.frame.DataFrame

In [41]:
cv_fit.vocabulary_.get('python')

6209

In [42]:
#Classement des mots les plus fréquents

cv_sum = cv_transform_title.sum(axis=0)
cv_words_freq = [(word, cv_sum[0, idx]) for word, idx in cv_fit.vocabulary_.items()]
cv_words_freq =sorted(cv_words_freq, key = lambda x: x[1], reverse=True)
cv_words_freq[:10]

[('using', 3088),
 ('file', 2062),
 ('use', 1531),
 ('error', 1483),
 ('net', 1477),
 ('android', 1367),
 ('java', 1296),
 ('python', 1279),
 ('data', 1051),
 ('window', 987)]

In [28]:
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import glob


#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

2023-04-14 19:09:34.665612: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-14 19:09:34.781298: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-14 19:09:35.225731: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-14 19:09:35.624984: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.

In [35]:
X_train['Title'][0][0:90]

'calculate rolling idxmax'

In [55]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words_titles = gen_words(X_train['Title'])

print (data_words_titles[0:5])

[['how', 'exactly', 'remove', 'punctuation', 'using', 'package'], ['how', 'debug', 'bat', 'script'], ['how', 'get', 'utc', 'time', 'midnight', 'given', 'timezone'], ['meteor', 'native', 'osx', 'window', 'app'], ['any', 'quick', 'win', 'make', 'net', 'remoting', 'faster', 'single', 'machine']]


In [60]:
id2word_title = corpora.Dictionary(data_words_titles)

corpus = []
for text in data_words_titles:
    new = id2word_title.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word_title[[0][:1][0]]
print (word)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]
exactly


In [50]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word_title,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [51]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
24    -0.512409  0.159428       1        1  13.929950
1      0.176637 -0.441375       2        1   5.935472
0      0.411278  0.204126       3        1   4.987247
25     0.314306  0.310036       4        1   4.783271
19     0.435701 -0.173927       5        1   4.750131
21    -0.018947 -0.462992       6        1   4.653225
29    -0.248257  0.383722       7        1   4.103870
16     0.421426  0.053594       8        1   3.948035
12    -0.376609 -0.208144       9        1   3.830251
27    -0.416532 -0.043763      10        1   3.749160
13    -0.286987  0.236548      11        1   3.566154
28    -0.137149 -0.373322      12        1   3.460658
4     -0.268786 -0.326370      13        1   3.249884
17     0.350019 -0.029505      14        1   3.195049
11    -0.016259  0.431937      15        1   3.012774
18     0.283696 -0.290592      16        1   2.974310
23     0.244678 -0.163276      17        1   2.958024
22    -0.310565  0.096017      18        1   2.931202
2     -0.272138 -0.128336      19        1   2.740996
3      0.085956  0.355381      20        1   2.687120
15     0.190677  0.333162      21        1   2.572981
5     -0.128475  0.303792      22        1   2.481213
7     -0.011090  0.208948      23        1   2.331739
14     0.094196 -0.250749      24        1   1.803391
6      0.169301  0.099416      25        1   1.772841
20    -0.107815 -0.168470      26        1   1.442742
26     0.018656 -0.161285      27        1   1.143650
9     -0.107053  0.034778      28        1   0.821933
8      0.014495  0.005669      29        1   0.119899
10     0.008049  0.005552      30        1   0.062827, topic_info=          Term          Freq         Total Category  logprob  loglift
1          how  22597.000000  22597.000000  Default  30.0000  30.0000
5        using   8211.000000   8211.000000  Default  29.0000  29.0000
339        use   3644.000000   3644.000000  Default  28.0000  28.0000
32        file   4079.000000   4079.000000  Default  27.0000  27.0000
74        what   3700.000000   3700.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
23        make      0.009282    887.975784  Topic30  -9.7328  -4.0960
24         net      0.009282   3194.881221  Topic30  -9.7328  -5.3764
27      single      0.009282    368.111088  Topic30  -9.7328  -3.2155
28         win      0.009282    133.390888  Topic30  -9.7328  -2.2004
29   algorithm      0.009282    324.198177  Topic30  -9.7328  -3.0885

[1373 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
220      12  0.967848   abstract
89        5  0.997935     access
174      12  0.998676        add
285      12  0.966402     advice
29        7  0.993220  algorithm
...     ...       ...        ...
254       1  0.996993      write
145      27  0.996827      xcode
449       5  0.997675        xml
340      26  0.773519      xpath
398      13  0.985470       zoom

[292 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[25, 2, 1, 26, 20, 22, 30, 17, 13, 28, 14, 29, 5, 18, 12, 19, 24, 23, 3, 4, 16, 6, 8, 15, 7, 21, 27, 10, 9, 11])

In [57]:
data_words = gen_words(X_train['text'])
print (data_words[0][0:20])

['how', 'exactly', 'remove', 'punctuation', 'using', 'package', 'update', 'think', 'may', 'workaround', 'solve', 'problem', 'add', 'one', 'code', 'dtms', 'dtm', 'remove', 'sparse', 'character']


In [61]:
id2word_text = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word_text.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word_text[[0][:1][0]]
print(word)

[(0, 3), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 2), (7, 2), (8, 2), (9, 1), (10, 3), (11, 17), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 4), (19, 2)]
abap


In [62]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word_text,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [1]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

NameError: name 'pyLDAvis' is not defined